<h1>Grafo delle co-authorship tra gli articoli del corso di informatica</h1>
<p>Dal dataset degli articoli di informatica vengono filtrati solamente quelli dei professori che attualmente sono nella lista del suddetto dipartimento.</p>

In [41]:
from matplotlib.font_manager import font_scalings
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

<h2>Caricamento dei dati</h2>

<h3>Dataset degli articoli di informatica</h3>
<p>In formato csv e convertito a dataframe</p>

In [42]:
# Carico il dataset
df_informatica = pd.read_csv("../data/informatica.csv")
df_informatica

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Mari, M.; Snidaro, L.","Mari, Marco (59388337700); Snidaro, Lauro (650...",59388337700; 6507499895,Survey of Neural Network Approaches to Target ...,2026,Information Fusion,127,NaN,103789,NaN,...,NaN,NaN,NaN,English,Inf. Fusion,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105017548985
1,"Mari, M.; Snidaro, L.","Mari, Marco (59388337700); Snidaro, Lauro (650...",59388337700; 6507499895,Ensemble of KalmanNets with innovation-based a...,2026,Information Fusion,127,NaN,103777,NaN,...,NaN,NaN,NaN,English,Inf. Fusion,Article,Final,NaN,Scopus,2-s2.0-105017546340
2,"Ceschia, S.; Di Gaspero, L.; Rosati, R.M.; Sch...","Ceschia, Sara (35193496000); Di Gaspero, Luca ...",35193496000; 6505944235; 57406536600; 6701629145,Multi-neighborhood simulated annealing for the...,2026,International Transactions in Operational Rese...,33,1,NaN,38,...,NaN,NaN,NaN,English,Int. Trans. Oper. Res.,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85211480324
3,"Pagliarini, R.","Pagliarini, Roberto (25229039800)",25229039800,Differential Flux-Balance Analysis Infers Meta...,2026,Lecture Notes in Computer Science,16051 LNCS,NaN,NaN,155,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-105022894987
4,"Madni, H.A.; Shujat, H.; de Nardin, A.; Zottin...","Madni, Hussain Ahmad (57195220347); Shujat, Ha...",57195220347; 59705771800; 57259162600; 5795942...,FsBAD: Data-efficient feature reconstruction f...,2026,Pattern Recognition Letters,199,NaN,NaN,113,...,NaN,NaN,NaN,English,Pattern Recogn. Lett.,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105021263351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,"Beŕtoni, A.; Mauri, G.; Sabadini, N.","Beŕtoni, Alberto (7005822404); Mauri, Giancarl...",7005822404; 7004284991; 6602383233,Equivalence and membership problems for regula...,1982,Lecture Notes in Computer Science,140 LNCS,NaN,NaN,61,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85037042961
4439,"Berry, D.M.; Ghezzi, C.; Mandrioli, D.; Tisato...","Berry, Daniel M. (7402256761); Ghezzi, Carlo (...",7402256761; 16512874900; 6603733719; 59777078600,Language constructs for real-time distributed ...,1982,Computer Languages,7,1,NaN,11,...,NaN,COLAD,NaN,English,Comput Lang,Article,Final,NaN,Scopus,2-s2.0-0020003724
4440,"Beŕtoni, A.; Mauri, G.; Sabadlni, N.","Beŕtoni, Alberto (7005822404); Mauri, Giancarl...",7005822404; 7004284991; 57203385151,A characterization of the class of functions c...,1981,Proceedings of the Annual ACM Symposium on The...,NaN,NaN,NaN,168,...,9798400715105; 0897910176; 9781605588179; 9781...,NaN,NaN,English,Proc. Annu. ACM Symp. Theory Comput.,Conference paper,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85049612709
4441,"Mauri, G.; Sabadini, N.","Mauri, Giancarlo (7004284991); Sabadini, Nicol...",7004284991; 6602383233,A # P-complete problem over arithmetical trees,1981,Lecture Notes in Computer Science,112 LNCS,NaN,NaN,308,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85034452402


<h3>Creazione della lista dei professori di informatica</h3>
<p>I nomi sono stati copiati dal sito dell'università e incollati in un file di testo.</p>

In [43]:
# Carico la lista dei professori dal file txt
with open("../data/professori_informatica.txt", "r", encoding="utf-8") as f:
    professori_informatica = [riga.strip() for riga in f]

with open("../data/professori_matematica.txt", "r", encoding="utf-8") as f:
    professori_matematica = [riga.strip() for riga in f]

with open("../data/professori_scienze_tecnologie_multimediali.txt", "r", encoding="utf-8") as f:
    professori_tecnologie_multimediali = [riga.strip() for riga in f]

def normalize_prof(name):
    cognome, nome = [x.strip() for x in name.split(',')]
    return f"{cognome} {nome}"

def normalize_list(lista_professori):
    lista_professori = set(normalize_prof(n) for n in lista_professori)
    return list(lista_professori)


professori_informatica = normalize_list(professori_informatica)
professori_matematica = normalize_list(professori_matematica)
professori_tecnologie_multimediali = normalize_list(professori_tecnologie_multimediali)

#print(professori_informatica)

#print("Prof di matematica: \n", professori_matematica)

print("Prof di tecnologie multimediali: \n", professori_tecnologie_multimediali)

Prof di tecnologie multimediali: 
 ['Cevolin Guglielmo', 'Liessi Davide', 'Toma Andrea', 'Rotondi Elena', 'Farinosi Manuela', 'Della Mea Vincenzo', 'Snidaro Lauro', 'Piciarelli Claudio', 'Fusiello Andrea', 'Fasino Dario', 'Ballis Demis', 'Drioli Carlo', 'Roitero Kevin', 'Raggiotto Francesco', 'Foresti Gian Luca', 'Piccolo Chiara', 'Zucco Rodolfo', 'Martinel Niki', 'Cossettini Luca', 'Rossitti Marco', 'Mizzaro Stefano', 'Calabretto Roberto', 'Chittaro Luca', 'Marinelli De Marco Giuseppe', 'Comini Marco', 'Micheloni Christian', 'Fontana Federico', 'Salvati Daniele', 'Atzori Paolo Cesare', 'Burigat Stefano', 'Di Iorio Marcela Beatriz']


<h2>Funzione per estrarre gli autori dalla relativa colonna del dataframe</h2>
<p>Gli autori sono memorizzati come: "Cognome, Nome (ID); ..."<br>
Quindi vogliamo trasformarli in "Cognome Nome"
</p>

In [44]:
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        # prendo la parte prima di "("
        a = a.split("(")[0].strip()
        
        # separo Cognome, Nome
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori


<h2>Creazione del grafo</h2>
<p>Viene anche salvato in formato pickle in modo tale che possa essere utilizzato in altri notebook</p>

In [45]:
G_prof_informatici = nx.Graph()
G_prof_matematica = nx.Graph()
G_prof_tecnologie= nx.Graph()

def creaGrafoProf(G,lista, path_pickle):
    for autori_string in df_informatica["Author full names"]:
        lista_autori = pulisci_autori(autori_string)

        # Filtra solo i professori del dipartimento
        autori_prof = [a for a in lista_autori if a in lista]

        # Aggiungi edge tra tutte le coppie di professori che hanno co-firmato
        for i in range(len(autori_prof)):
            for j in range(i + 1, len(autori_prof)):
                a1, a2 = autori_prof[i], autori_prof[j]

                if G.has_edge(a1, a2):
                    G[a1][a2]["weight"] += 1
                else:
                    G.add_edge(a1, a2, weight=1)
    with open(path_pickle, "wb") as f:
        pickle.dump(G, f)

creaGrafoProf(G_prof_informatici,professori_informatica ,"./grafi_pickle/articoli_informatica/G_prof_informatici.pkl")
creaGrafoProf(G_prof_matematica, professori_matematica,"./grafi_pickle/articoli_informatica/G_prof_matematica.pkl")
creaGrafoProf(G_prof_tecnologie, professori_tecnologie_multimediali,"./grafi_pickle/articoli_informatica/G_prof_tecnologie.pkl")

<h3>Dati del grafo</h3>
<ul>
    <li>Numero di nodi (professori coinvolti)</li>
    <li>Numero di archi (co-authorship)</li>
    <li>Per ogni professore i relativi collaboratori</li>
</ul>

In [46]:
def stampaDatiGrafico(g):
    print("Numero di nodi (professori coinvolti):", g.number_of_nodes())
    print("Numero di archi (co-autorship):", g.number_of_edges())

    # Mostra i vicini di un professore, se vuoi
    for n in g.nodes():
        print(n, "co-autore con:", list(g.neighbors(n)))
#stampaDatiGrafico(G_prof_informatici)
#stampaDatiGrafico(G_prof_matematica)
stampaDatiGrafico(G_prof_tecnologie)


Numero di nodi (professori coinvolti): 18
Numero di archi (co-autorship): 28
Piciarelli Claudio co-autore con: ['Foresti Gian Luca', 'Toma Andrea', 'Martinel Niki', 'Micheloni Christian', 'Snidaro Lauro']
Foresti Gian Luca co-autore con: ['Piciarelli Claudio', 'Toma Andrea', 'Fontana Federico', 'Fusiello Andrea', 'Drioli Carlo', 'Salvati Daniele', 'Farinosi Manuela', 'Micheloni Christian', 'Martinel Niki', 'Snidaro Lauro', 'Chittaro Luca']
Toma Andrea co-autore con: ['Piciarelli Claudio', 'Foresti Gian Luca', 'Martinel Niki', 'Drioli Carlo', 'Salvati Daniele']
Roitero Kevin co-autore con: ['Mizzaro Stefano', 'Della Mea Vincenzo']
Mizzaro Stefano co-autore con: ['Roitero Kevin', 'Della Mea Vincenzo', 'Fusiello Andrea']
Della Mea Vincenzo co-autore con: ['Mizzaro Stefano', 'Roitero Kevin']
Fontana Federico co-autore con: ['Foresti Gian Luca']
Martinel Niki co-autore con: ['Micheloni Christian', 'Toma Andrea', 'Foresti Gian Luca', 'Piciarelli Claudio']
Micheloni Christian co-autore con: [

<h3>Visualizzazione del grafo tramite libreria Pyvis</h3>
<p>Il file viene salvato in formato .html nella relativa cartella</p>

In [47]:
# Crea un grafo pyvis
def creaGrafo(G,output_path):
    net = Network(height="750px", width="100%", bgcolor="black", font_color="white", notebook=True, cdn_resources="remote", filter_menu=True)

    # Forza fisica per un layout più bello
    net.force_atlas_2based()

    # Aggiungi nodi e archi da NetworkX a PyVis
    for node in G.nodes():
        net.add_node(
            node,
            label=node,
            title=node,
            color="#4A90E2",   # colore nodi
            shape="dot",
            size=15
        )

    for u, v, data in G.edges(data=True):
        weight = data.get("weight", 1)
        net.add_edge(
            u,
            v,
            value=weight,
            title=f"Co-authored papers: {weight}"
        )
    # Salva in HTML
    net.write_html(output_path)
    #print("File generato:", output_path)
    return net


In [48]:

output_path_informatica= "../html/grafi_co-authorship/articoli_informatica/professori-informatica.html"
creaGrafo(G_prof_informatici, output_path_informatica).show(output_path_informatica)

../html/grafi_co-authorship/articoli_informatica/professori-informatica.html


In [49]:
output_path_matematica= "../html/grafi_co-authorship/articoli_informatica/professori-matematica.html"
creaGrafo(G_prof_matematica, output_path_matematica).show(output_path_matematica)

../html/grafi_co-authorship/articoli_informatica/professori-matematica.html


In [50]:
output_path_tecnologie= "../html/grafi_co-authorship/articoli_informatica/professori-tecnologie-multimediali.html"
creaGrafo(G_prof_tecnologie, output_path_informatica).show(output_path_tecnologie)

../html/grafi_co-authorship/articoli_informatica/professori-tecnologie-multimediali.html
